In [1]:
# The code was removed by Watson Studio for sharing.

# IBM Data Science Capstone Course: week 3 assignment

Importing libs

In [2]:
import urllib
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [3]:
#typing the Wikipedia url
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

Now it is time to parse the html using BeautifulSoup

In [4]:
#and parsing with BeautifulSoup
html = urllib.request.urlopen(url).read()
soup = BeautifulSoup(html,'lxml')

One list for each column of the dataframe is created and populated as we scroll by the parsed html

In [5]:
#now I find every table body in my soup
s = (soup.find_all("tbody"))
#the table of my interest is the first here
table = s[0]
i = 0
j = 0
# now I create 3 lists, one for each column of my dataframe
postcodeList = []
boroughList = []
neighList = []
#now I navigate through the tree to obtain the values of the table
for node in table.contents:
    if node != '\n':
        lineList = []
        j = 0
        for node2 in node.contents:
            if node2 != '\n':
                if i> 0:
                    if j == 0:
                        postcodeList.append(node2.text.strip())
                    elif j == 1:
                        boroughList.append(node2.text.strip())
                    else:
                        neighList.append(node2.text.strip())
                j = j+1
        i = i+1

Now the pandas dataframe is created

In [6]:
#and now I create my dataframe
d = {'PostalCode': postcodeList, 'Borough': boroughList, 'Neighborhood' : neighList}
df_postal = pd.DataFrame(data = d)
#and rearrange the columns order
df_postal = df_postal[['PostalCode','Borough','Neighborhood']]
df_postal.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In the next step a new copy of the dataframe is created without the not assigned boroughs

In [7]:
#dropping rows with not assigned borough
df_postal = df_postal[df_postal["Borough"] != "Not assigned"]
df_postal.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


For the not assigned neighborhoods, the borough values are copied into it

In [8]:
#replacing neighborhood values to borough value when the former is not assigned
df_postal["Neighborhood"].replace("Not assigned",df_postal["Borough"],inplace=True)
df_postal.head(10)

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Queen's Park
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


In the following steps, the neighborhoods are grouped and the resulting strings are treated and the dataframe is adjusted to its desired form

In [9]:
#concatenating neighborhoods for the same postal code/borough
df_postal = pd.DataFrame(data = df_postal.groupby(['PostalCode','Borough'])['Neighborhood'].agg(lambda x: "{%s}" % ', '.join(x)))
df_postal.head()

,,Neighborhood
PostalCode,Borough,
M1B,Scarborough,"{Rouge, Malvern}"
M1C,Scarborough,"{Highland Creek, Rouge Hill, Port Union}"
M1E,Scarborough,"{Guildwood, Morningside, West Hill}"
M1G,Scarborough,{Woburn}
M1H,Scarborough,{Cedarbrae}


In [10]:
#cleaning neighborhood column
df_postal["Neighborhood"] = df_postal["Neighborhood"].map(lambda x: x.lstrip('{').rstrip('}'))
#when the dataframe was grouped, PostalCode and Borough became indexes, let's create an unique index for the dataframe
df_postal.reset_index(level=['PostalCode','Borough'], inplace=True)
#and finally the dataframe
df_postal

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


And in the last step the dataframe shape is shown

In [11]:
df_postal.shape

(103, 3)

In [12]:
project.save_data(data=df_postal.to_csv(index=False),file_name='TorontoPostal.csv',overwrite=True)

{'asset_id': '4719715d-51b3-47e5-a4c4-dd60f8b29b74',
 'bucket_name': 'capstoneproject-donotdelete-pr-52n3kykujkxymr',
 'file_name': 'TorontoPostal.csv',
 'message': 'File TorontoPostal.csv has been written successfully to the associated OS'}